In [1]:
#!/usr/bin/env python
__author__ = "Shweta Patwa, Danyu Sun"

import csv
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
# import seaborn as sns; sns.set_theme(color_codes=True)
import sys
import textwrap
import time

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

# plt.style.use('seaborn-deep')

# Plot histogram on $A_i$ in $D$ and $D_s$ for comparison

In [2]:
def plot_both_hist(df, df_D_s, A_i, dom_A_i):
    # https://stackoverflow.com/questions/6871201/plot-two-histograms-on-single-chart-with-matplotlib
    plt.figure(figsize=(15, 6))
    bins = np.linspace(start = 0, stop = max(max(df[A_i].tolist()), max(df_D_s[A_i].tolist())) + 1)
    plt.hist([df[A_i], df_D_s[A_i]], bins, label=['$D$', '$D_s$'])
    plt.legend(loc='best')
    plt.show()

# (Non-private) Functions to compute:
- Count query q
- Sum query q
- Median query q

In [3]:
# Selection predicate is conjunctive
def helper_apply_pred(data_df, q):
    df_for_q = data_df
    for k, v in q.items():
        for clause in v:
            ineq = clause[0]
            if ineq == '<':
                df_for_q = df_for_q[df_for_q[k] < clause[1]]
            elif ineq == '<=':
                df_for_q = df_for_q[df_for_q[k] <= clause[1]]
            elif ineq == '>':
                df_for_q = df_for_q[df_for_q[k] > clause[1]]
            elif ineq == '>=':
                df_for_q = df_for_q[df_for_q[k] >= clause[1]]
            elif ineq == '==':
                df_for_q = df_for_q[df_for_q[k] == clause[1]]
            elif ineq == '!=':
                df_for_q = df_for_q[df_for_q[k] != clause[1]]
            else:
                print("Check query!!!")
    return df_for_q

# For a counting query at a time (selection predicate is conjunctive)
def get_query_result(data_df, q):
    return helper_apply_pred(data_df, q).shape[0]

# For sum with selection  (selection predicate is conjunctive)
def get_sum(data_df, q, A_i):
    df_for_q = helper_apply_pred(data_df, q)
    return df_for_q[A_i].sum()

# Assume - median is the elem in A_i with rank m
def get_median(data_df, q, A_i):
    df_for_q = helper_apply_pred(data_df, q)
    m = math.ceil(df_for_q.shape[0]/2)
    return sorted(list(df_for_q[A_i]))[m - 1]

---
# Read $D$ and $D_s$
- $D$ is derived from the IPUMS-CPS data
- $D_s$ generated using PrivBayes from SDGym

In [4]:
# https://github.com/yuchaotao/Private-Explanation-System/blob/main/data/ipums/explore.ipynb
code_dict = {'RELATE': {101: 'Head/householder',
    201: 'Spouse',
    202: 'Opposite sex spouse',
    203: 'Same sex spouse',
    301: 'Child',
    303: 'Stepchild',
    501: 'Parent',
    701: 'Sibling',
    901: 'Grandchild',
    1001: 'Other relatives, n.s.',
    1113: 'Partner/roommate',
    1114: 'Unmarried partner',
    1116: 'Opposite sex unmarried partner',
    1117: 'Same sex unmarried partner',
    1115: 'Housemate/roomate',
    1241: 'Roomer/boarder/lodger',
    1242: 'Foster children',
    1260: 'Other nonrelatives',
    9100: 'Armed Forces, relationship unknown',
    9200: 'Age under 14, relationship unknown',
    9900: 'Relationship unknown',
    9999: 'NIU'},
    'SEX': {1: 'Male', 2: 'Female', 9: 'NIU'},
    'RACE': {100: 'White',
    200: 'Black',
    300: 'American Indian/Aleut/Eskimo',
    650: 'Asian or Pacific Islander',
    651: 'Asian only',
    652: 'Hawaiian/Pacific Islander only',
    700: 'Other (single) race, n.e.c.',
    801: 'White-Black',
    802: 'White-American Indian',
    803: 'White-Asian',
    804: 'White-Hawaiian/Pacific Islander',
    805: 'Black-American Indian',
    806: 'Black-Asian',
    807: 'Black-Hawaiian/Pacific Islander',
    808: 'American Indian-Asian',
    809: 'Asian-Hawaiian/Pacific Islander',
    810: 'White-Black-American Indian',
    811: 'White-Black-Asian',
    812: 'White-American Indian-Asian',
    813: 'White-Asian-Hawaiian/Pacific Islander',
    814: 'White-Black-American Indian-Asian',
    815: 'American Indian-Hawaiian/Pacific Islander',
    816: 'White-Black--Hawaiian/Pacific Islander',
    817: 'White-American Indian-Hawaiian/Pacific Islander',
    818: 'Black-American Indian-Asian',
    819: 'White-American Indian-Asian-Hawaiian/Pacific Islander',
    820: 'Two or three races, unspecified',
    830: 'Four or five races, unspecified',
    999: 'Blank'},
    'MARST': {1: 'Married, spouse present',
    2: 'Married, spouse absent',
    3: 'Separated',
    4: 'Divorced',
    5: 'Widowed',
    6: 'Never married/single',
    7: 'Widowed or Divorced',
    9: 'NIU'},
    'CITIZEN': {1: 'Born in U.S',
    2: 'Born in U.S. outlying',
    3: 'Born abroad of American parents',
    4: 'Naturalized citizen',
    5: 'Not a citizen',
    9: 'NIU'},
    'WORKLY': {0: 'NIU',
    1: 'No',
    2: 'Yes'},
    'CLASSWKR': {0: 'NIU',
    10: 'Self-employed',
    13: 'Self-employed, not incorporated',
    14: 'Self-employed, incorporated',
    20: 'Works for wages or salary',
    21: 'Wage/salary, private',
    22: 'Private, for profit',
    23: 'Private, nonprofit',
    24: 'Wage/salary, government',
    25: 'Federal government employee',
    26: 'Armed forces',
    27: 'State government employee',
    28: 'Local government employee',
    29: 'Unpaid family worker',
    99: 'Missing/Unknown'},
    'EDUC': {0: 'NIU or no schooling',
    1: 'NIU or blank',
    2: 'None or preschool',
    10: 'Grades 1, 2, 3, or 4',
    11: 'Grade 1',
    12: 'Grade 2',
    13: 'Grade 3',
    14: 'Grade 4',
    20: 'Grades 5 or 6',
    21: 'Grade 5',
    22: 'Grade 6',
    30: 'Grades 7 or 8',
    31: 'Grade 7',
    32: 'Grade 8',
    40: 'Grade 9',
    50: 'Grade 10',
    60: 'Grade 11',
    70: 'Grade 12',
    71: '12th grade, no diploma',
    72: '12th grade, diploma unclear',
    73: 'High school diploma or equivalent',
    80: '1 year of college',
    81: 'Some college but no degree',
    90: '2 years of college',
    91: "Associate's degree, occupational/vocational program",
    92: "Associate's degree, academic program",
    100: '3 years of college',
    110: '4 years of college',
    111: "Bachelor's degree",
    120: '5+ years of college',
    121: '5 years of college',
    122: '6+ years of college',
    123: "Master's degree",
    124: 'Professional school degree',
    125: 'Doctorate degree',
    999: 'Missing/Unknown'}
}

In [5]:
df = pd.read_csv("./2011_2019_D.csv")
col_names = list(df.columns)
print(col_names)

rel = list(code_dict['RELATE'].values())
age = [i for i in range(0, 80)] + [80, 85] # https://cps.ipums.org/cps-action/variables/AGE#codes_section
sex = list(code_dict['SEX'].values())
rac = list(code_dict['RACE'].values())
mar = list(code_dict['MARST'].values())
cit = list(code_dict['CITIZEN'].values())
wor = list(code_dict['WORKLY'].values())
cla = list(code_dict['CLASSWKR'].values())
edu = list(code_dict['EDUC'].values())
inc = [0, 500000] # https://cps.ipums.org/cps-action/variables/INCTOT#codes_section

df_D_s = pd.read_csv("./2011_2019_D_s.csv")
df_D_s['AGE'] = df_D_s['AGE'].astype(int)
df_D_s['INCTOT'] = df_D_s['INCTOT'].astype(int)

['RELATE', 'AGE', 'SEX', 'RACE', 'MARST', 'CITIZEN', 'CLASSWKR', 'EDUC', 'WORKLY', 'INCTOT']


---
# Variables $\epsilon, \tau, \beta$
---

# Detect FP/FN

In [6]:
def FP_FN(re, q_D, q_D_s, tau):
    if (re == 0 and (q_D_s - tau < q_D and q_D < q_D_s + tau)):
        return 'FN'
    if (re == 1 and (q_D <= q_D_s - tau or q_D >= q_D_s + tau)):
        return 'FP'

# $LM_{sum}$

In [7]:
def lmsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, f_handle):
    re = -1
    
    q_D = get_sum(df, q, A_i)
    q_D_s = get_sum(df_D_s, q, A_i)
    l = q_D_s - tau
    r = q_D_s + tau
    
    f_handle.write("(I = (%s, %s), Truth = %s, eps = %s) Algo returns:\n" 
          %(l, r, "Distance bound satisfied" if (l < q_D and q_D < r) else "Distance bound unmet", eps))
    # ----------------------------------------------------------------------------------------------------------------
    GS_Q = max(dom_A_i) - min(dom_A_i)
    nu_q = np.random.laplace(scale = GS_Q/eps)
    f_handle.write("\tDP estimate = %s + %s = %s\n" %(q_D, nu_q, q_D + nu_q))
    
    if -1*tau < q_D + nu_q - q_D_s and q_D + nu_q - q_D_s < tau:
        f_handle.write("Distance bound satisfied\n")
        re = 1
    else:
        f_handle.write("Distance bound unmet\n")
        re = 0
    return re, q_D, q_D_s

In [8]:
def err_lmsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, f_handle):
    start_time = time.time()
    
    FN = 0
    FP = 0
    for i in range(100):
        re, q_D, q_D_s = lmsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, f_handle)
        tmp = FP_FN(re, q_D, q_D_s, tau)
        if tmp == 'FN':
            FN += 1
        if tmp == 'FP':
            FP += 1
    err = (FN + FP)/100
    f_handle.write("\n")
    
    print("---- %s seconds ----" % (time.time() - start_time))
    
    return err

# $R2T_{sum}$

In [9]:
def r2tsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, beta, f_handle):
    re = -1
    
    q_D = get_sum(df, q, A_i)
    q_D_s = get_sum(df_D_s, q, A_i)
    l = q_D_s - tau
    r = q_D_s + tau
    
    f_handle.write("(I = (%s, %s), Truth = %s, eps = %s) Algo returns:\n" 
          %(l, r, "Distance bound satisfied" if (l < q_D and q_D < r) else "Distance bound unmet", eps))
    # ----------------------------------------------------------------------------------------------------------------
    GS_Q = max(dom_A_i) - min(dom_A_i)
    log_GS_Q = math.ceil(math.log(GS_Q, 2))
    
    Q = []
    trunc_thres = []
    for i in range(1, log_GS_Q + 1):
        new_conjunct = copy.deepcopy(q)
        if A_i in new_conjunct:
            new_conjunct[A_i].append(['<=', 2**i])
        else:
            new_conjunct[A_i] = [['<=', 2**i]]
#         print("(R2T) %s" %(new_conjunct))
        
        Q.append(new_conjunct)
        trunc_thres.append(2**i)
    if log_GS_Q == 0:                                # Binary domain
        new_conjunct = copy.deepcopy(q)
        if A_i in new_conjunct:
            new_conjunct[A_i].append(['<=', 2])
        else:
            new_conjunct[A_i] = [['<=', 2]]
#         print("(R2T) %s" %(new_conjunct))
        
        Q.append(new_conjunct)
        trunc_thres = [2]
    f_handle.write("Truncation thresholds: %s\n" %(trunc_thres))
    
    max_across_Q = 0
    for i in range(len(Q)):
        q_i = get_sum(df, Q[i], A_i)
        GS_q_i = trunc_thres[i]                      # tau^j
        f_handle.write("\tIter# %s: q(D) = %s with truncation threshold of %s\n" %(i, q_i, GS_q_i))
        
        nu_q_i = np.random.laplace(scale = log_GS_Q * GS_q_i / eps)
        penalty = (log_GS_Q * math.log(log_GS_Q/beta) * GS_q_i) / eps
        noisy_q_i = q_i + nu_q_i - penalty
        f_handle.write("\t\t(eqn 7, noisy estimate) %.3f\n" %(noisy_q_i))
        
        if noisy_q_i > max_across_Q:
            max_across_Q = noisy_q_i
    
    f_handle.write("(noisy)%.3f - %.3f = %.3f\n" %(max_across_Q, q_D_s, max_across_Q - q_D_s))
    if -1*tau < max_across_Q - q_D_s and max_across_Q - q_D_s < tau:
        f_handle.write("Distance bound satisfied\n")
        re = 1
    else:
        f_handle.write("Distance bound unmet\n")
        re = 0
    return re, q_D, q_D_s

In [10]:
def err_r2tsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, beta, f_handle):
    start_time = time.time()
    
    FN = 0
    FP = 0
    for i in range(100):
        re, q_D, q_D_s = r2tsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, beta, f_handle)
        tmp = FP_FN(re, q_D, q_D_s, tau)
        if tmp == 'FN':
            FN += 1
        if tmp == 'FP':
            FP += 1
    err = (FN + FP)/100
    f_handle.write("\n")
    
    print("---- %s seconds ----" % (time.time() - start_time))
    
    return err

# $SVT_{sum}$

## - Find $t_j$ to stop at in $SVT_{sum}$

In [11]:
def private_bound_DS(q, A_i, GS_Q, df, eps, theta, f_handle):
    nu_q = np.random.laplace(scale = 1/(eps/3))
    tilde_n = get_query_result(df, q) + nu_q
    
    rho = np.random.laplace(scale = 1/(eps/3))
    
    for i in range(1, math.ceil(math.log(GS_Q, 2)) + 1):
        new_conjunct = copy.deepcopy(q)
        if A_i in new_conjunct:
            new_conjunct[A_i].append(['<=', 2**i])
        else:
            new_conjunct[A_i] = [['<=', 2**i]]
#         print("(Private bound) %s" %(new_conjunct))
        
        nu_q_c = np.random.laplace(scale = 1/(eps/3))
        q_c = get_query_result(df, new_conjunct)
        
        if q_c + nu_q_c >= theta*tilde_n + rho:
            return i
    return math.ceil(math.log(GS_Q, 2))

In [12]:
# nu_q_i: queries are monotonic - "all queries whose answers are different change in the same direction"
def SVT_ith_q(eps_2, df, q_i, GS_q_i, T, rho, f_handle):
    nu_q_i = np.random.laplace(scale = 1/eps_2)
    noisy_q_i = q_i/GS_q_i + nu_q_i

    T_i = T/GS_q_i
    noisy_T_i = T_i + rho
    
    f_handle.write("\t\tCheck: %.3f (=%.3f + %.3f) >= %.3f (=%.3f + %.3f)?\n" %(noisy_q_i, q_i/GS_q_i, nu_q_i, 
                                                                                noisy_T_i, T_i, rho))
    return noisy_q_i, noisy_T_i

In [13]:
def svtsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, f_handle):
    re = -1
    
    q_D = get_sum(df, q, A_i)
    q_D_s = get_sum(df_D_s, q, A_i)
    l = q_D_s - tau
    r = q_D_s + tau
    
    f_handle.write("(I = (%s, %s), Truth = %s, eps = %s) Algo returns:\n" 
          %(l, r, "Distance bound satisfied" if (l < q_D and q_D < r) else "Distance bound unmet", eps))
    # ----------------------------------------------------------------------------------------------------------------
    GS_Q = max(dom_A_i) - min(dom_A_i)
    
    Q = []
    trunc_thres = []
    for i in range(1, private_bound_DS(q, A_i, GS_Q, df, eps/3, 0.95, f_handle) + 1):
        new_conjunct = copy.deepcopy(q)
        if A_i in new_conjunct:
            new_conjunct[A_i].append(['<=', 2**i])
        else:
            new_conjunct[A_i] = [['<=', 2**i]]
#         print("(R2T) %s" %(new_conjunct))
        
        Q.append(new_conjunct)
        trunc_thres.append(2**i)
    if math.ceil(math.log(GS_Q, 2)) == 0:            # Binary domain
        new_conjunct = copy.deepcopy(q)
        if A_i in new_conjunct:
            new_conjunct[A_i].append(['<=', 2])
        else:
            new_conjunct[A_i] = [['<=', 2]]
#         print("(R2T) %s" %(new_conjunct))
        
        Q.append(new_conjunct)
        trunc_thres = [2]
    f_handle.write("Truncation thresholds: %s\n" %(trunc_thres))
    
    eps_1 = eps_2 = eps/3
    rho = np.random.laplace(scale = 1/eps_1)
        
    f_handle.write("Check if any sum >= r:\n")
    for i in range(len(Q)):
        q_i = get_sum(df, Q[i], A_i)
        GS_q_i = trunc_thres[i]
        f_handle.write("\tIter# %s: q(D) = %s with truncation threshold of %s\n" %(i, q_i, GS_q_i))
        
        noisy_q_i, noisy_T_i = SVT_ith_q(eps_2, df, q_i, GS_q_i, r, rho, f_handle)
        if noisy_q_i >= noisy_T_i:
            f_handle.write("Distance bound unmet\n")
            re = 0
            return re, q_D, q_D_s
    
    f_handle.write("Check if any sum >= l+1:\n")
    for i in range(len(Q)):
        q_i = get_sum(df, Q[i], A_i)
        GS_q_i = trunc_thres[i]
        f_handle.write("\tIter# %s: q(D) = %s with truncation threshold of %s\n" %(i, q_i, GS_q_i))
        
        noisy_q_i, noisy_T_i = SVT_ith_q(eps_2, df, q_i, GS_q_i, l+1, rho, f_handle)
        if noisy_q_i >= noisy_T_i:
            f_handle.write("Distance bound satisfied\n")
            re = 1
            return re, q_D, q_D_s
    
    f_handle.write("Distance bound unmet\n")
    re = 0
    return re, q_D, q_D_s

In [14]:
def err_svtsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, f_handle):
    start_time = time.time()
    
    FN = 0
    FP = 0
    for i in range(100):
        re, q_D, q_D_s = svtsum(q, A_i, dom_A_i, df, df_D_s, tau, eps, f_handle)
        tmp = FP_FN(re, q_D, q_D_s, tau)
        if tmp == 'FN':
            FN += 1
        if tmp == 'FP':
            FP += 1
    err = (FN + FP)/100
    f_handle.write("\n")
    
    print("---- %s seconds ----" % (time.time() - start_time))
    
    return err

---
---
# 9 queries (3 small, 3 medium, 3 large)

In [15]:
q1 = {'SEX': [['==', 'Female']], 'RACE': [['==', 'White-Black']], 'WORKLY': [['==', 'No']]}
q2 = {'RACE': [['==', 'Asian only']], 'MARST': [['==', "Separated"]], 'CITIZEN': [['==', 'Born in U.S']]} 
q3 = {'SEX': [['==', 'Male']], 'EDUC': [['==', "Professional school degree"]], 'MARST': [['==', 'Married, spouse absent']]} 

q6 = {'SEX': [['==', 'Male']], 'RACE': [['==', 'Asian only']], 'WORKLY': [['==', 'No']]}    
q5 = {'SEX': [['==', 'Female']], 'EDUC': [['==', "Master's degree"]], 'MARST': [['==', 'Widowed']]} 
q4 = {'RACE': [['==', 'White']], 'MARST': [['==', "Divorced"]], 'CITIZEN': [['==', 'Not a citizen']]} 

q7 = {'SEX': [['==', 'Male']], 'RACE': [['==', 'Black']], 'WORKLY': [['==', 'No']]}
q8 = {'SEX': [['==', 'Female']], 'EDUC': [['==', "High school diploma or equivalent"]], 'MARST': [['==', 'Never married/single']]}
q9 = {'RACE': [['==', 'White']], 'MARST': [['==', "Married, spouse present"]], 'CITIZEN': [['==', 'Born in U.S']]}

queries = [q1, q2, q3, q4, q5, q6, q7, q8, q9]

In [16]:
def print_(q):
    print('DS_qd = %s, q(D) = %s, q(D_s) = %s'%(helper_apply_pred(df, q)['INCTOT'].max(), 
                                                get_sum(df, q,'INCTOT'), 
                                                get_sum(df_D_s, q,'INCTOT')))

In [17]:
num = 1
for i in queries:
    print('q'+ str(num) + ":")
    print_(i)
    num = num + 1

q1:
DS_qd = 127764, q(D) = 6915340, q(D_s) = 6942866
q2:
DS_qd = 359892, q(D) = 7653748, q(D_s) = 7818555
q3:
DS_qd = 439420, q(D) = 9891949, q(D_s) = 10129562
q4:
DS_qd = 500000, q(D) = 123543040, q(D_s) = 128497757
q5:
DS_qd = 384842, q(D) = 138131505, q(D_s) = 139557517
q6:
DS_qd = 278011, q(D) = 143179279, q(D_s) = 149069654
q7:
DS_qd = 276799, q(D) = 327036909, q(D_s) = 336130017
q8:
DS_qd = 403353, q(D) = 685635093, q(D_s) = 690711885
q9:
DS_qd = 500000, q(D) = 23542765109, q(D_s) = 23434676868


# Experiments

In [18]:
tau_frac = [0.002, 0.008, 0.032, 0.128, 0.512]   
default_eps = 0.25

eps = [0.0625, 0.125, 0.25, 0.5, 1]   
default_tau = 0.032

delta = 0.05

In [20]:
lst_LM_vary_tau = [[], [], [], [], [], [], [], [], []]
lst_LM_vary_eps = [[], [], [], [], [], [], [], [], []]

lst_R2T_vary_tau = [[], [], [], [], [], [], [], [], []]
lst_R2T_vary_eps = [[], [], [], [], [], [], [], [], []]

lst_SVT_vary_tau = [[], [], [], [], [], [], [], [], []]
lst_SVT_vary_eps = [[], [], [], [], [], [], [], [], []]

# LM---vary tau

In [21]:
f_handle = open('sum_LM_vary_tau.txt', 'w')
num = 0
for i in queries:
    f_handle.write("%s\n" %(i))
    q_D_s = get_sum(df_D_s, i, 'INCTOT')
    lst_LM_vary_tau[num].append(err_lmsum(i, 'INCTOT', inc, df, df_D_s, tau_frac[0]*q_D_s, default_eps, f_handle))
    lst_LM_vary_tau[num].append(err_lmsum(i, 'INCTOT', inc, df, df_D_s, tau_frac[1]*q_D_s, default_eps, f_handle))
    lst_LM_vary_tau[num].append(err_lmsum(i, 'INCTOT', inc, df, df_D_s, tau_frac[2]*q_D_s, default_eps, f_handle))
    lst_LM_vary_tau[num].append(err_lmsum(i, 'INCTOT', inc, df, df_D_s, tau_frac[3]*q_D_s, default_eps, f_handle))
    lst_LM_vary_tau[num].append(err_lmsum(i, 'INCTOT', inc, df, df_D_s, tau_frac[4]*q_D_s, default_eps, f_handle))
    print()
    num = num + 1
f_handle.close()

---- 30.97059202194214 seconds ----
---- 31.209147930145264 seconds ----
---- 32.15785598754883 seconds ----
---- 32.87772798538208 seconds ----
---- 33.88369083404541 seconds ----

---- 16.089548110961914 seconds ----
---- 16.53566288948059 seconds ----
---- 15.956474304199219 seconds ----
---- 16.155670166015625 seconds ----
---- 16.75202989578247 seconds ----

---- 34.16968393325806 seconds ----
---- 37.65628099441528 seconds ----
---- 40.932899713516235 seconds ----
---- 41.81365418434143 seconds ----
---- 41.2616868019104 seconds ----

---- 56.98295879364014 seconds ----
---- 44.35149312019348 seconds ----
---- 42.45921778678894 seconds ----
---- 42.37126183509827 seconds ----
---- 42.7848699092865 seconds ----

---- 37.25869011878967 seconds ----
---- 47.63454008102417 seconds ----
---- 46.53449988365173 seconds ----
---- 44.6331741809845 seconds ----
---- 45.11708617210388 seconds ----

---- 42.17567992210388 seconds ----
---- 43.51933526992798 seconds ----
---- 43.9058189392089

In [22]:
print(lst_LM_vary_tau)

[[0.0, 0.96, 0.89, 0.62, 0.17], [0.02, 0.01, 0.86, 0.67, 0.17], [0.0, 0.04, 0.82, 0.5, 0.04], [0.01, 0.06, 0.26, 0.0, 0.0], [0.09, 0.24, 0.16, 0.0, 0.0], [0.0, 0.02, 0.26, 0.0, 0.0], [0.0, 0.03, 0.21, 0.0, 0.0], [0.08, 0.44, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [], [], []]
